# 7章 ロジスティクスネットワークの最適設計を行う10本ノック

ここでは、最適化計算を行ういくつかのライブラリを用いて、最適化計算を実際に行っていきます。  
そして、前章で用いたネットワーク可視化などの技術を駆使し、計算結果の妥当性を確認する方法についても学んでいきます。

### ノック６１：輸送最適化問題を解いてみよう

In [8]:
import numpy as np
import pandas as pd
from itertools import product
from pulp import LpVariable, lpSum, value
from ortoolpy import model_min, addvars, addvals

# Data の読み込み
df_tc = pd.read_csv('trans_cost.csv', index_col='工場')
df_demand = pd.read_csv('demand.csv')
df_supply = pd.read_csv('supply.csv')

# 初期設定
np.random.seed(1)
nw = len(df_tc.index)
nf = len(df_tc.columns)
pr = list(product(range(nw), range(nf)))

# 数理 Model 作成
m1 = model_min()  # 最小化を行う Model

v1 = {(i, j): LpVariable('v%d_%d' % (i, j), lowBound=0) for i, j in pr}  #LpVariable を用いて主役となる変数 v1 をdict形式で与える。
m1 += lpSum(df_tc.iloc[i][j] * v1[i, j] for i, j in pr) # lpSum によって、各輸送 Route の Cost を格納した DataFrame と、v1 とのそれぞれの要素の積の和によって目的関数を定義。

for i in range(nw):
    m1 += lpSum(v1[i, j] for j in range(nf)) <= df_supply.iloc[0][i]  # 制約条件: 工場の製造する製品が需要量を満たす
for j in range(nf):
    m1 += lpSum(v1[i, j] for i in range(nw)) >= df_demand.iloc[0][j]  # 制約条件: 倉庫の供給する部品が供給限界を超過しないように

m1.solve()  # 変数 v1 が最適化され、最適な総輸送 Cost が求められる。

# 総輸送 Cost計算
df_tr_sol = df_tc.copy()
total_cost = 0
for k, x in v1.items():
    i, j = k[0], k[1]
    df_tr_sol.iloc[i][j] = value(x)
    total_cost += df_tc.iloc[i][j] * value(x)

print(df_tr_sol)
print(f'総輸送 Cost: {str(total_cost)}')

    F1  F2  F3  F4
工場                
W1  28   7   0   0
W2   0   0  31   5
W3   0  22   0  20
総輸送 Cost: 1296.0


最適化計算の結果、総輸送 Cost は、1,296(万円)という結果になりもともとの総輸送 Cost: 1,433(万円)と比較し、大きく Cost 削減ができた。

### ノック６２：最適輸送ルートをネットワークで確認しよう

### ノック６３：最適輸送ルートが制約条件内に収まっているかどうかを確認しよう

### ノック６４：生産計画に関するデータを読み込んでみよう

### ノック６５：利益を計算する関数を作ってみよう

### ノック６６：生産最適化問題を解いてみよう

### ノック６７：最適生産計画が制約条件内に収まっているかどうかを確認しよう

### ノック６８：ロジスティクスネットワーク設計問題を解いてみよう

### ノック６９：最適ネットワークにおける輸送コストとその内訳を計算しよう

### ノック７０：最適ネットワークにおける生産コストとその内訳を計算しよう